In [1]:
import json
import uuid
from pymongo import MongoClient

In [2]:
db_dict = {
    "id": "evmirna",
    "title": "EVmiRNA",
    "url": "http://bioinfo.life.hust.edu.cn/EVmiRNA/",
    "description": "EVmiRNA is a database of miRNA profiling in extracellular vesicles",
    "basicInfo": "Extracellular vesicles (EVs) released by living cells include exosomes and microvesicles (MVs), which contain various molecules from parental cells and are potential sources for disease diagnostic and therapeutic biomarkers. miRNA is the most well-studied molecule type in EVs because of its important functions and biomarker properties. Thus, we build the Extracellular Vesicles miRNA database (EVmiRNA) to collect comprehensive miRNA expression profiles in EVs. In EVmiRNA database, we analyzed 462 smRNA sequencing datasets of EVs from 17 tissues/diseases. The miRNA expression profiles, miRNA regulated pathways, miRNA function, miRNA related drugs and publications are showed to support the miRNA biomarker discovery.",
    "categories": ["miRNA", "extracellular vesicles", "disease"],
    "species": ["Homo Sapiens"],
    "updatedAt": "2019-08-30 11:11:11"
}

In [3]:
class MongoMir:
    __mongo = MongoClient("mongodb://username:passwd@ip:port/dbname")

    def __init__(self, col_name = 'mirinfo'):
        self.__col_name = col_name
    
    def get_data(self, output={}, condition={}):
        output['_id'] = 0
        mcur = self.__mongo.mirnasnp[self.__col_name].find(
            condition, output, no_cursor_timeout=True
        )
        return mcur.count()

    def get_mirnas(self):
        mcur = self.__mongo.EVmiRNA.mir_annotation.find(
            {}, {'_id': 0, 'miRNA_id': 1}
        )
        res = [item['miRNA_id'] for item in mcur]
        return res


In [4]:
class ENTRY(object):
    def __init__(self, type, title, url):
        self.id = str(uuid.uuid4())
        self.type = type
        self.title = title
        self.url = url
        self.dbId = "evmirna"
        self.updatedAt = "2019-08-30 11:11:11"
        self.description = ""
        self.basicInfo = ""
        self.species = ["Homo Sapiens"]
        self.attrs = {
            "symbol": title,
        }
    def __getattr__(self, attr):
        return self[attr]

In [5]:
def get_entry(it, type = 'miRNA ID'):
    url = f'http://bioinfo.life.hust.edu.cn/EVmiRNA#!/miRNA_info?miRNA={it}'
    e = ENTRY(type, it, url)

    return json.dumps(e.__dict__)

In [6]:
mongo_mirnasnp = MongoMir()
mirna_ids = mongo_mirnasnp.get_mirnas()

In [7]:
with open('/home/liucj/index.bs', 'w') as fh:
    header = 'DB' + '\t' + json.dumps(db_dict) + '\n'
    fh.write(header)

    for it in mirna_ids:
        line = 'ENTRY' + '\t' + get_entry(it = it, type = 'miRNA ID') + '\n'

        fh.write(line)